# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: Inteligencia Artificial y Aprendizaje Automático**

Tecnológico de Monterrey

Prof Luis Eduardo Falcón Morales

**Actividad de la Semana 5**

### **Modelos basados en Árboles**



**Nombres y matrículas de los integrantes del Equipo:**

*   Nombre y matrícula
*   Nombre y matrícula
*   Nombre y matrícula



# **PARTE - 1 - Bosque Aleatorio (Random Forest) - Clasificación**

In [ ]:
# Importamos lo necesario para la actividad

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, RandomizedSearchCV, learning_curve, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.metrics import make_scorer, recall_score, accuracy_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier

from imblearn.pipeline import Pipeline  # Observa que usamos imblearn.Pipeline en lugar de sklearn
from imblearn.over_sampling import SMOTE


# Semilla para reproducibilidad
np.random.seed(17)

In [ ]:
# Para esta actividad vamos a generar datos sintéticos para un problema de
# clasificación binario utilizando "make_classification" de sklearn.

# Recuerda consultar la documentación para mayor información:
# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html

# Utilizaremos los siguientes calores de los hiperparámetros de make_classification:
# - n_samples: número de muestras (10,000)
# - n_features: número total de características (20)
# - n_informative: número de características informativas (14)
# - n_redundant: número de características redundantes (6) .. .incluímos algunas redundantes
# - weights: pesos para las clases [0,1]-->[0.9, 0.1] para conseguir el desbalance 90%-10%
# - class_sep: separación entre clases (mayor valor --> clases más separables y menos complejo)
# - flip_y: fracción de ejemplos cuya clase se cambia aleatoriamente (ruido), para hacerlo más complejo
# - random_state: semilla para reproducibilidad

X, y = make_classification(
    n_samples=10_000,          # 10,000 registros
    n_features=20,             # 20 factores en total
    n_informative=14,          # 14 factores informativos
    n_redundant=6,             # 6 factores redundantes (dependientes)
    weights=[0.9, 0.1],        # Desbalance de clases: 90% clase 0, 10% clase 1
    class_sep=1.0,             # Separación entre clases
    n_classes=2,               # Dos clases
    n_clusters_per_class=1,    # Si queremos agregar complejidad adicional > 1
    flip_y=0.03,               # Añadir algo de ruido. default 0.01
    random_state=17,
)

In [ ]:
# Por el momento generaremos un conjunto de dato que supondremos ya
# están escalados y todos las variables son numéricas, para concentrarnos
# en el modelo de Bosque Aleatrorio.

# Escalamos las características para que estén en el mismo rango:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Creamos un DataFrame para su mejor manejo
feature_names = [f'feature_{i+1}' for i in range(20)]
df = pd.DataFrame(X_scaled, columns=feature_names)
df['target'] = y

print(f"Total de registros generados: {len(df)}")
print(f"Distribución de clases: {df['target'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'}")
print(f"Cantidad de features: {len(feature_names)}")

In [ ]:
pd.DataFrame(df).describe().T   # Observamos que todos los factores varían en el mismo
                                # rango de aproximadamente -4 y 4.

In [ ]:
# Separamos las variables de entrada y la variable objetivo de salida:
X = df.drop('target', axis=1).values
y = df['target'].values

# Dividir el conjunto de datos en entrenamiento (80%) y prueba (20%)
# Como vamos a utilizar Validación Cruzada, haremos la partición
# en Entrenamiento y Prueba.
# Además usamos s"tratify" para mantener la proporción de clases en la partición.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17, stratify=y)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]} muestras")

In [ ]:
# Hagamos esta partición temporal para tener un valor aproximado del desempeño
# mínimo que alcanzará nuestro modelo más simple.
Xt, Xv, yt, yv = train_test_split(X_train, y_train, test_size=0.2, random_state=17, stratify=y_train)

estrategias = ['most_frequent','prior','stratified','uniform']

for estrategia in estrategias:
  dummy_clf = DummyClassifier(strategy=estrategia, random_state=17)
  dummy_clf.fit(Xt, yt)
  y_pred = dummy_clf.predict(Xv)

  # Tabla para almacenar resultados
  results = []

  # "pos_label" indica la clase con respecto a la cual evaluar cada métrica.
  acc = accuracy_score(yv, y_pred)
  rec = recall_score(yv, y_pred, pos_label=1)
  prec = precision_score(yv, y_pred, pos_label=1)
  f1_sc = f1_score(yv, y_pred, pos_label=1)

  results.append({'Accuracy': acc,
                'Recall': rec,
                'Precision': prec,
                'F1 Score': f1_sc
                })
  print(f"Estrategia: {estrategia}")
  print(f"Accuracy: {acc:.4f}")
  print(f"Recall: {rec:.4f}")
  print(f"Precision: {prec:.4f}")
  print(f"F1 Score: {f1_sc:.4f}")
  print()

## **Ejercicio - 1**

### **En este ejercicio deseamos obtener el umbral del desempeño mínimo que debiera alcanzar nuestro modelo, es decir, obtener el desempeño del modelo más simple (dummy). Consideraremos las siguientes políticas de la función DummyClassifier(): "most_frequent", "prior, "stratified" y "uniform".**

Recuerda revisar la documentación correspondiente:

https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html

### **Ejercicio 1a**

#### **Para los casos "most_frequent" y "prior" observamos que se obtiene un "UndefinedMetricWarning" y nos dice que la métrica Precision no está bien definida ("ill-defined") ¿Qué significa este aviso? ¿Y si usamos la fórmula de Precision, qué nos resultaría?**

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Incluye aquí tus comentarios.

None

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

### **Ejercicio 1b**

#### **Supongamos que la métrica que vamos a estar monitoreando es el "F1-score". Si esta fuera nuestra decisión y considerando los valores numéricos obtenidos en la celda anterior, ¿cuál de las cuatro políticas ("most_frequent", "prior, "stratified", "uniform") recomendarías utilizar para obtener el desempeño mínimo que debiera tener nuestro mejor modelo que vamos a construir con RandomForest? Y por lo tanto, ¿cuál sería este valor mínimo? Justifica tu decisión.**

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Incluye aquí tus comentarios.

None

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## **Ejercicio - 2:**

* #### **En lo que resta de esta primera parte de la Actividad, supondremos que la métrica F1-score es la que nos interesa monitorear.**

* #### **Así, a continuación deberás encontrar la mejor configuración del modelo Bosque Aleatorio que te resulte en la mejor métrica F1-score con respecto a la clase positiva 1.**

* #### **Además, el modelo no debe estar sub-entrenado o sobre-entrenado con respecto a esta métrica F1-score.**

* #### **Deberas decidir si se requiere incluir alguna técnica de sub-mestreo y/o sobre-muestro.**

* #### **Incluye los hiperparámetros que consideres adecuados, pero recuerda que si incluyes demasiados, el tiempo de entrenamiento se incrementa.**


Revisa la documentación correspondiente:

* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

* https://imbalanced-learn.org/stable/references/over_sampling.html

* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

In [ ]:
# Ejercicio 2:


# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# +++++++ INICIA SECCIÓN PARA INCLUIR TUS AJUSTES ++++++++++++++++++

# Incluyo algunos ejemplos, pero puedes incluir más si lo deseas, revisa
# la documentación correspondiente.

# Definimos nuestro pipeline:
pipeline = Pipeline([
    #('smote', SMOTE()),  # Descomenta si deseas usar técnica de balanceo
    ('model', RandomForestClassifier(random_state=17))
])

# Definimos los posibles valores para la búsqueda de malla.
# El total de opciones a buscar en esta malla se obtiene con el producto
# de la cantidad de casos de cada hiperparámetro.
# Observa la diferencia entre el guión bajo doble y el sencillo.
param_grid = {
    'smote__k_neighbors': [5,7],  # Descomenta para usar hiperparámetros de la técnica de balanceo.
    'model__n_estimators': [50,100],  # Hiperparámetros del modelo ...
    #'model__max_depth': [],
    #'model__min_samples_split': [],
    #'model__class_weight':[]

    # agregar todos los demás hiperparámetros que desees...
}


# Utilizaremos Validación Cruzada Estratificada:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

# Aquí definimos la métrica a utilizar, en nuestro caso, F1-score:
scorer = make_scorer(f1_score, average='binary', pos_label=1) # Esta línea no la modifiques.

# Conjuntamos todo en la búsqueda de malla GridSearch:
grid_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    n_iter=None,   # Aquí indicas el número de opciones del param_grid en los que harás la búsqueda.
    cv=cv,
    scoring=scorer,
    n_jobs=-1,
    verbose=1
)

# +++++++++++++ TERMINA SECCIÓN PARA REALIZAR AJUSTES +++++++++++++++
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++



# Hacemos el ajuste del modelo con los datos de entrenamiento:
grid_search.fit(X_train, y_train)

# Y evaluamos con el mejor  conjunto de prueba
#best_model = grid_search.best_estimator_
#y_pred = best_model.predict(X_test)

best_params = grid_search.best_params_
print(f"\nMejores parámetros encontrados:\n {best_params}\n\n")




# **************************************************************************
# Gráfico de curvas de aprendizaje del mejor modelo.
best_model = grid_search.best_estimator_

# Definimos tamaños de entrenamiento relativos al conjunto de entrenamiento:
train_sizes = np.linspace(0.1, 1.0, 10)

# Calculamos curvas de aprendizaje con cross-validation:
train_sizes, train_scores, valid_scores = learning_curve(
    estimator=best_model,     # Usamos el mejor modelo encontrado
    X=X_train,
    y=y_train,
    train_sizes=train_sizes,  # Tamaños de entrenamiento a evaluar
    cv=5,
    scoring='f1',             # Métrica a evaluar, en nuetro caso F1-score
    n_jobs=-1,                # Usar todos los núcleos disponibles
    random_state=17
)

# Calculamos medias y desviaciones estándar:
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
valid_mean = np.mean(valid_scores, axis=1)
valid_std = np.std(valid_scores, axis=1)


# Área sombreada en el gráfico para la desviación estándar:
plt.figure(figsize=(6, 5))
plt.fill_between(train_sizes, train_mean - train_std,
                 train_mean + train_std, alpha=0.1, color='blue')
plt.fill_between(train_sizes, valid_mean - valid_std,
                 valid_mean + valid_std, alpha=0.1, color='orange')

# Grafcamos el polígono de las medias:
plt.plot(train_sizes, train_mean, 'o-', color='blue', label='F1-score de Entrenamiento')
plt.plot(train_sizes, valid_mean, 'o-', color='orange', label='F1-score de Validación (Cruzada)')


plt.title(f'Curvas de Aprendizaje del mejor modelo')
plt.xlabel('Tamaño del conjunto de entrenamiento')
plt.ylabel('métrica F1-score')
plt.grid(True)
plt.legend(loc='lower right')
#plt.ylim([0.8, 1.01])  # Puedes ajustar el rango del eje Y según tus datos
plt.show()

In [ ]:
grid_search   # mejor configuración obtenida

## **Conjunto de Prueba**

In [ ]:
# Pasemos a predecir con el conjunto de Prueba (Test) una vez
# que encontraste tu mejor modelo.

y_pred = best_model.predict(X_test)

# Probabilidades de predicción para la clase 1
y_proba = best_model.predict_proba(X_test)[:, 1]

# Matriz de confusión:
cm = confusion_matrix(y_test, y_pred, normalize='true')
plt.figure(figsize=(3,3))
sns.heatmap(cm, annot=True, fmt='.2g', cmap='Blues', cbar=False)   # en caso de enteros: fmt='d'
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.xticks([0.5, 1.5], ['Clase 0', 'Clase 1'])
plt.yticks([0.5, 1.5], ['Clase 0', 'Clase 1'])
plt.show()

In [ ]:
# Reporte de clasificación estándar
print("Reporte de Clasificación Estándar:")
print(classification_report(y_test, y_pred))

## **Nuevo umbral de decisión con F1-score**

In [ ]:
# En problemas desbalanceados, el umbral por defecto de 0.5 puede no ser el óptimo
# para hacer las predicciones:
# Si y_proba>0.5, entonces lo asignamos a la Clase_1, en otro caso, a la Clase_0.

# Vamos a encontrar el umbral que maximiza el F1-score y determinar si sigue
# siendo el valor por defecto.

thresholds = np.arange(0.1, 0.9, 0.05)
f1_scores = []

for threshold in thresholds:
    y_binary = (y_proba >= threshold).astype(int)
    f1 = f1_score(y_test, y_binary)
    f1_scores.append(f1)

# Encontrar el mejor umbral
best_threshold_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_idx]
best_f1 = f1_scores[best_threshold_idx]

plt.figure(figsize=(8,4))
plt.plot(thresholds, f1_scores, 'o-', color='purple')
plt.axvline(x=best_threshold, color='r', linestyle='--',
            label=f'Umbral óptimo = {best_threshold:.2f}, F1 = {best_f1:.3f}')
plt.axvline(x=0.5, color='g', linestyle='--',
            label=f'Umbral predeterminado = 0.5, F1 = {f1_score(y_test, (y_proba >= 0.5).astype(int)):.3f}')
plt.title('F1-score vs Umbral de Decisión')
plt.xlabel('Umbral')
plt.ylabel('F1-score')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Evaluar el modelo con el umbral óptimo
y_pred_optimal = (y_proba >= best_threshold).astype(int)
print("\nResultados con umbral óptimo:")
print(classification_report(y_test, y_pred_optimal))

In [ ]:
# Matriz de confusión con umbral óptimo
cm_optimal = confusion_matrix(y_test, y_pred_optimal, normalize='true')
plt.figure(figsize=(3,3))
sns.heatmap(cm_optimal, annot=True, fmt='.2g', cmap='Blues', cbar=False)
plt.title(f'Matriz de Confusión (Umbral = {best_threshold:.2f})')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.xticks([0.5, 1.5], ['Clase 0', 'Clase 1'])
plt.yticks([0.5, 1.5], ['Clase 0', 'Clase 1'])
plt.show()

## **Ejercicio - 3**

#### **Con base a los resultados obtenidos responde los siguientes incisos que ayuden a concluir esta primera parte de la actividad.**

* **Ejercicio 3a: Comenta por qué el modelo final que obtuviste no está subentrenado, ni sobreentrenado.**


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Incluye aquí tus comentarios.

None

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

* **Ejercicio 3b: Comenta las diferencias (si las hay) que observas entre usar el umbral predeterminado 0.5 y el nuevo umbral.**


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Incluye aquí tus comentarios.

None

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


* **Ejercicio 3c: Comenta el impacto que viste al usar o no alguna técnica de submuestreo.**


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Incluye aquí tus comentarios.

None

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


* **Ejercicio 3d: incluye tus comentarios finales de esta primera parte de la actividad.**

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Incluye aquí tus comentarios.

None

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


# **PARTE - 2 - XGBoost - Regressor**

Ver documentación para hiperparámetros del modelo:

https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor

https://xgboost.readthedocs.io/en/stable/parameter.html

In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from xgboost import XGBRegressor
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt
import seaborn as sns

# Semilla para reproducibilidad
np.random.seed(17)

In [ ]:
# Generamos un dataset de regresión con 10,000 muestras y 20 características
X, y = make_regression(n_samples=10_000,
                       n_features=20,
                       n_informative=15,
                       n_targets=1,
                       noise=100.,
                       random_state=17)

# Convertimos a DataFrame de Pandas:
df = pd.DataFrame(X, columns=[f"feat_{i}" for i in range(X.shape[1])])
df['target'] = y

print("Forma del dataset:", df.shape)


In [ ]:
df.describe().T   # Observa que estos factores ya están en un rango análogo de -4 a 4, aprox.

In [ ]:
# División en Train vs Test (80% vs 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

print("Tamaño de entrenamiento:", X_train.shape)
print("Tamaño de test:", X_test.shape)

## **Ejercicio - 4**

### **Desempeño del modelo base (baseline)**

#### **Las líneas de código de la siguiente celda son un análisis que nos ayudarán posteriormente a determinar si el modelo que obtengamos estará o no subentrenado.**

* **Ejercicio 4a: Explica con tus palabras de manera clara lo que hacen estas líneas de código para poder obtener de ahí el modelo base (baseline) de un modelo de regresión.**


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Incluye aquí tus comentarios.

None

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++



* **Ejercicio 4b: Explica el significado de los valores numéricos mostrados: Valor_1 y Valor_2.**


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Incluye aquí tus comentarios.

None

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
Xt, Xv, yt, yv = train_test_split(X_train, y_train, test_size=0.2, random_state=17)
yt_mean = np.mean(yt)
y_pred_baseline = np.full(shape=yv.shape, fill_value=yt_mean)
rmse_baseline = np.sqrt(mean_squared_error(yv, y_pred_baseline))

print(f"Valor_1-Ejercicio-4b: {yt_mean:.4f}\n")
print(f"Valor_2-Ejercicio-4b: {rmse_baseline:.4f}")

## **Ejercicio - 5**

* #### **Encuentra la mejor configuración del modelo XGBoost que te resulte con la métrica RMSE.**

* #### **Además, el modelo no debe estar sub-entrenado o sobre-entrenado.**

* #### **Incluye los hiperparámetros que consideres adecuados, pero recuerda que si incluyes demasiados, el tiempo de entrenamiento se incrementa.**

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# +++++++ INICIA SECCIÓN PARA INCLUIR TUS AJUSTES ++++++++++++++++++

# Instanciamos el modelo base:
model = XGBRegressor(random_state=17, n_jobs=-1)

param_grid = {
    'n_estimators': [100, 200],
    # Incluye aquí todos los casos que desees buscar en la malla.
}

# Métricas de regresión a evaluar:
scoring = {
    'MAE': 'neg_mean_absolute_error',
    'RMSE': 'neg_root_mean_squared_error',
    'R2': 'r2',
    'MAPE': 'neg_mean_absolute_percentage_error'
}


# Configuración del grid search aleatorio:
grid_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=None,         # Indica la cantidad de casos a buscar en la malla.
    scoring=scoring,
    refit='RMSE',      # Selecciona el mejor modelo según esta métrica RMSE.
    cv=5,
    n_jobs=-1,
    verbose=1
)

# +++++++++++++ TERMINA SECCIÓN PARA REALIZAR AJUSTES +++++++++++++++
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++



# Pasamos al entrenamiento del modelo:
grid_search.fit(X_train, y_train)


# **********************************************************************
# Medimos el desempeño del modelo con respecto al modelo base (baseline):
rmse_xgb = -grid_search.best_score_

print(f"\nRMSE del modelo XGBoost: {rmse_xgb:.4f}\n")
print(f"Resultado-para-el-Ejercicio-6b: {(rmse_baseline - rmse_xgb) / rmse_baseline * 100:.1f}%\n")



# ***********************************************************************
# Visualizamos el aprendizaje del mejor modelo:
# Usamos el mejor modelo encontrado por GridSearchCV
best_model = grid_search.best_estimator_

# Definimos los tamaños de entrenamiento a evaluar
train_sizes = np.linspace(0.1, 1.0, 10)

# Calcular las curvas de aprendizaje usando RMSE
train_sizes, train_scores, test_scores = learning_curve(
    estimator=best_model,
    X=X_train,
    y=y_train,
    train_sizes=train_sizes,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    shuffle=True,
    random_state=17
)

# Convertimos los puntajes negativos de RMSE a positivos
train_rmse = -train_scores.mean(axis=1)
test_rmse = -test_scores.mean(axis=1)

plt.figure(figsize=(8,4))
plt.plot(train_sizes, train_rmse, label='Entrenamiento (RMSE)', color='blue')
plt.plot(train_sizes, test_rmse, label='Validación (RMSE)', color='red', linestyle='--')
plt.title('Curva de Aprendizaje - RMSE')
plt.xlabel('Tamaño del conjunto de entrenamiento')
plt.ylabel('RMSE')
plt.legend()
plt.grid(True)
plt.show()

### **Obtenemos finalmente información con respecto al conjunto de Prueba:**

In [ ]:
grid_search   # configuración del mejor modelo encontrado

In [ ]:
# Predicciones en el conjunto de Prueba (Test) con el mejor modelo encontrado:
y_pred = grid_search.predict(X_test)

# Calculamos los valores de las métricas:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("\nResultados-para-el-Ejercicio-6c:")
print("\nMétricas en Test:")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
# Gráfico de dispersión entre valores reales y predichos
plt.figure(figsize=(8,4))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Valores reales')
plt.ylabel('Predicciones')
plt.title('Valores reales vs Predicciones')
plt.grid(True)
plt.show()

## **Ejercicio - 6**

#### **Con base a los resultados obtenidos responde los siguientes incisos que ayuden a concluir esta segunda parte de la actividad.**

* **Ejercicio 6a: Comenta por qué el modelo final que obtuviste no está subentrenado, ni sobreentrenado.**


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Incluye aquí tus comentarios.

None

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


* **Ejercicio 6b: Indica cómo interpretas el valor obtenido en "Resultado-para-el-Ejercicio-6b".**


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Incluye aquí tus comentarios.

None

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++



* **Ejercicio 6c: Indica cómo interpretas cada uno de los resultados que obtuviste en "Resultados-para-el-Ejercicio-6c".**


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Incluye aquí tus comentarios.

None

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

* **Ejercicio 3d: Incluye tus comentarios finales de esta segunda parte de la actividad.**

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Incluye aquí tus comentarios.

None

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## **Ejercicio - 7**

* **Ejercicio 7: incluye tus comentarios finales de esta actividad.**

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Incluye aquí tus comentarios.

None

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# **Fin de la Actividad de modelos basados en áboles**